In [1]:
import requests

api_key = "20211595:29e2994c-2337-48d4-a828-4a8f424edfd0"
endpoint = "https://iaapi.novaims.unl.pt/v1/completions"
def llama_completion(prompt, max_tokens=1024):
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "Gemma3:12b",
        "prompt": prompt,
        "max_tokens": max_tokens
    }
    response = requests.post(endpoint, headers=headers, json=data)
    return response.json()

def main():
    prompt = "Conta uma anedota sobre programadores"
    response = llama_completion(prompt)
    print(response["choices"][0]["text"])

if __name__ == "__main__":
    main()

Claro, aqui vai uma anedota sobre programadores que sempre me faz rir:

Dois programadores estão trabalhando em um projeto complexo. Um deles, o João, está com um problema particularmente difícil e está falando sozinho, frustrado:

"Por que isso não funciona? Eu declarei a variável, defini o tipo, fiz a chamada de função... Não pode ser!"

O outro programador, o Pedro, que estava do outro lado da sala, levantou a cabeça e perguntou:

"João, você está falando com o código de novo?"

João respondeu, sem tirar os olhos da tela: "Não, eu estou dialogando com ele. Ele me deve uma explicação."

Pedro balançou a cabeça sorrindo e disse: "Acho que você precisa de um café."

A anedota ilustra o quão profundo o relacionamento entre um programador e o código que ele escreve pode ser. É quase como se estivessem conversando diretamente com a máquina! 😂

Espero que tenha gostado!


**ESSENCIAL**

In [14]:
import os
from openai import AzureOpenAI
import api

endpoint = "https://ai-bcds.openai.azure.com/"
model_name = "gpt-4o-mini"
deployment = "gpt-4o-mini"

subscription_key = api.key
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)


**CHATBOT BASE sem ler documentos**

In [12]:
# === CHAT FUNCTION ===
def ask_financial_bot(user_input: str) -> str:
    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful and professional financial assistant. "
                "Always give clear, trustworthy financial advice, and explain things in a beginner-friendly way."
            )
        },
        {
            "role": "user",
            "content": user_input
        }
        
    ]

    response = client.chat.completions.create(
        model=deployment,
        messages=messages,
        max_tokens=1024,
        temperature=0.7,
        top_p=1.0
    )

    return response.choices[0].message.content

# === EXAMPLE USE ===
if __name__ == "__main__":
    question = input("Ask your financial assistant: ")
    answer = ask_financial_bot(question)
    print("\n💬 Answer:", answer)


Ask your financial assistant:  how do i invest in stocks



💬 Answer: Investing in stocks can be a rewarding way to build wealth over time, but it's important to understand the basics before getting started. Here’s a step-by-step guide to help you begin:

### 1. **Educate Yourself**
   - **Understand What Stocks Are:** Stocks represent ownership in a company. When you buy a stock, you own a small piece of that company.
   - **Learn Basic Terminology:** Familiarize yourself with terms like dividends, market capitalization, and P/E ratio.

### 2. **Set Your Investment Goals**
   - **Determine Your Objectives:** Are you investing for retirement, a major purchase, or just to grow your wealth? Your goals will influence your investment strategy.
   - **Assess Your Risk Tolerance:** Consider how much risk you are willing to take. Stocks can be volatile, and prices can fluctuate significantly.

### 3. **Choose an Investment Account**
   - **Brokerage Account:** You’ll need to open a brokerage account to buy and sell stocks. This can be done through tr

**CHATBOT a ler 1 documento**

In [18]:
import fitz  # PyMuPDF

def extract_text_from_pdf(path):
    doc = fitz.open(path)
    return "\n".join(page.get_text() for page in doc)

pdf_text = extract_text_from_pdf("docs/Documents for training and evaluation-20250507/Questions_Answers_Censored.pdf")

def ask_with_pdf_context(user_input):
    messages = [
        {
            "role": "system",
            "content": (
                "You are a financial assistant. Use the following document as context:\n\n"
                + pdf_text[:5000]  # Keep it short to avoid hitting token limits
            )
        },
        {"role": "user", "content": user_input}
    ]

    response = client.chat.completions.create(
        model=deployment,
        messages=messages,
        max_tokens=1024
    )
    return response.choices[0].message.content


In [19]:
# === EXAMPLE USE ===
if __name__ == "__main__":
    question = input("Ask your financial assistant: ")
    answer = ask_with_pdf_context(question)
    print("\n💬 Answer:", answer)


Ask your financial assistant:  O Fidelidade Savings é mais indicado para que perfil de risco?



💬 Answer: O Fidelidade Savings é adequado para diferentes perfis de risco:

1. **Clientes conservadores**: Opção Seguro, que oferece capital e rendimento garantido.
2. **Clientes com tolerância moderada ao risco**: Opção Proteção, que garante 90% do capital investido no vencimento e possui exposição ao Índice Multiativos.
3. **Clientes propensos ao risco**: Opção Dinâmico, com 100% de exposição ao Índice Multiativos.
4. **Clientes preocupados com questões ambientais, sociais e de governança**: Opção ESG, que investe no índice Multiativos ESG, mas com risco associado.


**CHATBOT a ler vários documentos**

In [20]:
import fitz

def extract_text_from_pdf(path):
    doc = fitz.open(path)
    return "\n".join(page.get_text() for page in doc)

In [22]:
import os

pdf_dir = "docs/Documents for training and evaluation-20250507"
all_texts = []

for pdf_file in os.listdir(pdf_dir):
    if pdf_file.endswith(".pdf"):
        path = os.path.join(pdf_dir, pdf_file)
        text = extract_text_from_pdf(path)
        all_texts.append(text)


In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.create_documents(all_texts)
texts = [chunk.page_content for chunk in chunks]


In [29]:
from openai import AzureOpenAI
import numpy as np

embedding_client = AzureOpenAI(
    api_key="8J6pTdfaGgA5r193UVLsBshUspqwNpal42Jse1aHaok1cWNTLpRkJQQJ99BDACYeBjFXJ3w3AAABACOGLa23",
    api_version="2024-02-01",
    azure_endpoint="https://ai-bcds.openai.azure.com/"
)

def get_embedding(text):
    response = embedding_client.embeddings.create(
        input=[text],
        model="text-embedding-3-small"  # Use the deployment name you set up
    )
    return response.data[0].embedding

# Create embeddings for all text chunks
embeddings = [get_embedding(text) for text in texts]


In [30]:
import faiss

dimension = len(embeddings[0])
index = faiss.IndexFlatL2(dimension)

# Add embeddings to the FAISS index
index.add(np.array(embeddings).astype('float32'))

# Optionally, save the index for later use
faiss.write_index(index, "my_pdf_index.faiss")


ModuleNotFoundError: No module named 'faiss'

In [31]:
def get_top_k_chunks(query, k=3):
    query_vec = np.array([get_embedding(query)]).astype('float32')
    D, I = index.search(query_vec, k)  # Get top-k similar chunks
    return [texts[i] for i in I[0]]  # Return the corresponding text chunks

# Example usage
query = "What are the latest trends in financial advising?"
top_chunks = get_top_k_chunks(query)


NameError: name 'index' is not defined

In [28]:
%pip install faiss

ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss
Note: you may need to restart the kernel to use updated packages.
